Add your name and PID
<br> Kush Parmar A14032759<br> Alex Biz A14455796 <br> Jessica Cramer A12606890

## Introduction

### Overview

### Research Question

### Hypothesis

### Background

###  Dataset Description

## Data Cleaning and Pre-processing

### Setup

In [1]:
# import libraries
import pandas as pd
import spotipy
from profanity_check import predict, predict_prob
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# load in raw data
raw_data = pd.read_csv("./dataset/with_prof_no_norm.csv")
# raw_data.index = raw_data["index"]
# raw_data = raw_data.drop(columns=["index"])

# add popularity column
# raw_data["pop"] = np.zeros(len(raw_data))

raw_data

,Unnamed: 0,index,song,year,artist,genre,lyrics,pop,prof
0,0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\r\r\r\r\r\nYou know I'...",42.0,2
1,5,5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard\r\r\r\r\r\nChurch bells ringing\r\r\r\...,46.0,0
2,6,6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend\r\...,33.0,0
3,16,16,poison,2009,beyonce-knowles,Pop,You're bad for me I clearly get it\r\r\r\r\r\n...,39.0,2
4,18,18,beautiful-liar,2007,beyonce-knowles,Pop,Ay\r\r\r\r\r\nAy\r\r\r\r\r\nAy (Nobody likes t...,61.0,0
5,19,19,beautiful-liar-spanish,2007,beyonce-knowles,Pop,Ay! Ay!\r\r\r\r\r\n(Nobody likes being played)...,34.0,0
6,20,20,beautiful-liar-spanglish-version,2007,beyonce-knowles,Pop,Ay! Ay!\r\r\r\r\r\n(Nobody likes being played)...,33.0,0
7,24,24,bello-embustero,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\r\r\r\r\r\nOh! Beyonc, Beyonc\r\r\...",34.0,0
8,29,29,if,2007,beyonce-knowles,Pop,He is always laughin' and flirting with me\r\r...,68.0,0
9,31,31,bello-embustero-beautiful-liar-spanish-version,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\r\r\r\r\r\nOh! Beyonce, Beyonce\r\...",34.0,0


In [3]:
# get popularity data from spotify API

# Setup spotify API 
# client_manage = SpotifyClientCredentials(client_id="a47de82ee2f24629ba951b0d1beea37c" , client_secret="539e6aca7ae648b981d8dc494f276cd8")
# spot = spotipy.Spotify(client_credentials_manager=client_manage)

In [4]:
# iterate over all songs 
# for i in range(359999, len(raw_data)):
    # song = raw_data.iloc[i]["song"]
    # artist = raw_data.iloc[i]["artist"]
    
    # replace hyphen with spaces
    # song = song.replace("-", " ")
    # artist = artist.replace("-", " ")
    
    # get popularity ratings - maybe other stuff later on
    # result = spot.search("track: " + song + " artist: " + artist, limit=3)
    # result = result['tracks']['items']
    
    # go through list, and find song
    # if len(result) != 0:
        # name = result[0]["name"]
        # pop = result[0]["popularity"]
        # raw_data.at[i,"pop"] = pop

    # print
    # print("iteration " + str(i) +"/" + str(len(raw_data)), end="\r")
    
    # save progress for every song, and keep track of progress
    # if (i % 1000) == 0:
        # write to file
        # raw_data.to_csv("./dataset/with_pop.csv")
        
# write to file
# raw_data.to_csv("./dataset/with_pop.csv")

### Cleaning

In [11]:
# Drop NaN lyrics
# raw_data.dropna(inplace=True, subset=["lyrics"])

# Drop 0 popularity
# raw_data = raw_data.drop(raw_data[raw_data["pop"] == 0].index)

# clean \rs from all lyrics
def clean_lyrics(lyrics):
    lyrics = lyrics.replace("\r", "")
    return lyrics
raw_data["lyrics"] = raw_data["lyrics"].apply(clean_lyrics)
raw_data

,Unnamed: 0,index,song,year,artist,genre,lyrics,pop,prof
0,0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing?\nYou know I'm gonna cu...",42.0,2
1,5,5,all-i-could-do-was-cry,2009,beyonce-knowles,Pop,I heard\nChurch bells ringing\nI heard\nA choi...,46.0,0
2,6,6,once-in-a-lifetime,2009,beyonce-knowles,Pop,This is just another day that I would spend\nW...,33.0,0
3,16,16,poison,2009,beyonce-knowles,Pop,You're bad for me I clearly get it\nI don't se...,39.0,2
4,18,18,beautiful-liar,2007,beyonce-knowles,Pop,"Ay\nAy\nAy (Nobody likes to be played)\nOh, Be...",61.0,0
5,19,19,beautiful-liar-spanish,2007,beyonce-knowles,Pop,Ay! Ay!\n(Nobody likes being played)\nAy!\nOh!...,34.0,0
6,20,20,beautiful-liar-spanglish-version,2007,beyonce-knowles,Pop,Ay! Ay!\n(Nobody likes being played)\nAy!\nOh!...,33.0,0
7,24,24,bello-embustero,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\nOh! Beyonc, Beyonc\nOh! Beyonc, B...",34.0,0
8,29,29,if,2007,beyonce-knowles,Pop,He is always laughin' and flirting with me\nAn...,68.0,0
9,31,31,bello-embustero-beautiful-liar-spanish-version,2007,beyonce-knowles,Pop,"Ay! Ay! Ay!\nOh! Beyonce, Beyonce\nOh! Beyonce...",34.0,0


In [ ]:
# add profanity column

def prof(lyrics):
    lyrics = lyrics.split("\n")
    prof = 0
    for line in lyrics:
        prof += predict([line])[0]
    return prof

raw_data["prof"] = raw_data["lyrics"].apply(prof)

In [20]:
# raw_data.to_csv("./dataset/with_prof_no_norm.csv")

In [7]:
# add normalised column


["You're bad for me I clearly get it\r\r\r\r\r",
 "I don't see how something good could come from loving you\r\r\r\r\r",
 'The death of me must be your mission\r\r\r\r\r',
 "Cause with every hug and kiss you're snatching every bit of strength\r\r\r\r\r",
 "That I'm gon need to fight off the inevitable\r\r\r\r\r",
 "And it's a heart breaking situation I'm up in, but I can't control\r\r\r\r\r",
 '[Chorus]\r\r\r\r\r',
 "You're just like poison\r\r\r\r\r",
 'Slowly moving through my system\r\r\r\r\r',
 'Breaking all of my defenses with time\r\r\r\r\r',
 "You're just like poison and I just don't get it\r\r\r\r\r",
 'How can something so deadly feel so right?\r\r\r\r\r',
 "I'm not sure of what to do it's a catch 22\r\r\r\r\r",
 "cause the cure is found in you I don't want it but I do\r\r\r\r\r",
 "You're just like poison\r\r\r\r\r",
 "My affliction, I'm addicted, I can't lie\r\r\r\r\r",
 'Kiss me one more time before I die\r\r\r\r\r',
 '[Verse 2]\r\r\r\r\r',
 'You aint right take me high\r\r

## Data Analysis

In [1]:
# summary, and description of data

In [ ]:
# EDA